In [1]:
import os
%pwd
os.chdir('../')
%pwd

'/Users/al/langchain-projects/Text-Summarizer-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    
    

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name =config.tokenizer_name
        )
        return data_transformation_config

In [6]:
import os
import textSummarizer.logging as logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length = 1024,
            truncation = True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'],
                max_length = 128,
                truncation = True)
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
    }
    
    def convert(self):
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_examples_to_features, batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
        
    

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    logger.error(e)
    raise e

[2024-04-06 15:20:07,215: INFO: common: Read YAML file: config/config.yaml]
[2024-04-06 15:20:07,217: INFO: common: Read YAML file: params.yaml]
[2024-04-06 15:20:07,217: INFO: common: Directory already exists: artifacts]
[2024-04-06 15:20:07,217: INFO: common: Created directory: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/al/miniconda3/envs/textS/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 297128.32 examples/s]
